In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from datetime import timedelta
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.stats import skew, kurtosis
from matplotlib.colors import to_rgba

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/CFEM Greenmantle Project/'

Mounted at /content/drive


# Feature Extraction

In [3]:
path = '/content/drive/MyDrive/CFEM Greenmantle Project/'
ts = pd.read_csv(path+'Data/Data_adj/ts_minutely_return.csv')
# Convert the column to datetime with minute accuracy
ts = ts.drop(columns = ['HG.v.0', 'CL.v.0', 'ZT.v.0', 'RTY.v.0'])
ts = ts.rename(columns={'Unnamed: 0': 'datetime', 'ES.v.0': 'SP500', 'GC.v.0': 'Gold', 'ZN.v.0': 'Treasury10Y'})
ts['datetime'] = pd.to_datetime(ts['datetime']).dt.strftime('%Y-%m-%d %H:%M')
# ts = ts.fillna(0)
ts

,datetime,SP500,Treasury10Y,Gold
0,2017-07-09 22:02,-0.000052,0.000000,0.000206
1,2017-07-09 22:03,0.000000,0.000000,0.000041
2,2017-07-09 22:04,0.000155,0.000000,0.000000
3,2017-07-09 22:05,0.000000,0.000125,-0.000165
4,2017-07-09 22:06,-0.000103,-0.000063,0.000041
...,...,...,...,...
3772913,2024-09-10 23:55,-0.000227,0.000000,0.000039
3772914,2024-09-10 23:56,-0.000068,0.000135,0.000020
3772915,2024-09-10 23:57,0.000136,0.000000,0.000098
3772916,2024-09-10 23:58,0.000091,0.000068,0.000098


In [4]:
path = '/content/drive/MyDrive/CFEM Greenmantle Project/'

# event = pd.read_csv(path+"Data/Data_adj/event_filtered.csv")
event = pd.read_csv(path+'Data/Event Data/event_data_cleaned_ver_1027.csv')
event['Event_Time'] = pd.to_datetime(event['Event_Time']).dt.floor('min')
event = event.rename(columns={'Z_score_Surprise_rolling': 'Surprise'})

event.head()

,Event,Event_Time,Event_Type,Surprise,year
0,10-Year Note Auction,2014-10-08 13:00:00,Bond Auctions,0.000000,2014
1,10-Year Note Auction,2014-11-12 13:00:00,Bond Auctions,0.707107,2014
2,10-Year Note Auction,2015-06-10 13:00:00,Bond Auctions,1.150322,2015
3,10-Year Note Auction,2016-01-13 13:00:00,Bond Auctions,-0.759838,2016
4,10-Year Note Auction,2016-02-10 13:00:00,Bond Auctions,-1.613994,2016


In [5]:
# @title function: merge data around event
def merge_data_around_event(event_data, ts_data, window_unit="M", window_start=-60, window_end=60):
  '''This function merges the time series data with the event data.
  '''
  if window_unit not in ["H", "D", "M"]:
    raise ValueError("window_unit must be 'H' for hour, 'D' for day, or 'M' for minute.")
  if window_unit == "D":
    window_start = window_start*24*60
    window_end = window_end*24*60
  elif window_unit == "H":
    window_start = window_start*60
    window_end = window_end*60

  # Function to generate records for the given time range
  def generate_records(row, window_start, window_end):
      # Create the time range (minute intervals)
      time_range = pd.date_range(start=row['Event_Time'] + pd.Timedelta(minutes=window_start),
                    end=row['Event_Time'] + pd.Timedelta(minutes=window_end),
                    freq='min')

      # Convert the datetime to the desired string format: YYYY-mm-dd HH:MM
      time_range_str = time_range.strftime('%Y-%m-%d %H:%M')

      # Duplicate the event row for each time in the range
      expanded_data = pd.DataFrame({
          'Event': row['Event'],
          'Event_Type': row['Event_Type'],
          'Surprise': row['Surprise'],
          'Event_Time': row['Event_Time'],
          'datetime': time_range_str
      })

      return expanded_data

  expanded_event = pd.concat([generate_records(row, window_start, window_end) for _, row in event_data.iterrows()], ignore_index=True)
  merged = pd.merge(expanded_event, ts_data, on='datetime', how='left')

  merged1 = merged[(merged['datetime'] >= min(ts.index)) & (merged['datetime'] <= max(ts.index))]

  return merged1

# merged = merge_data_around_event(event, ts, "H", -6, 6)
# merged

In [6]:
# @title function: aggregate_features
from scipy.stats import skew, kurtosis
import pandas as pd

def aggregate_features(df, group_by_cols, agg_columns):
    """
    This function groups the input dataframe by specified columns and aggregates given numerical columns
    by calculating the mean, standard deviation, skewness, and kurtosis for each group.

    Parameters:
    df (DataFrame): The input dataframe.
    group_by_cols (list): The list of columns to group by.
    agg_columns (list): The list of numerical columns to aggregate.

    Returns:
    DataFrame: A dataframe with the aggregated results.
    """
    # Convert the Event_Time column to datetime format if not already
    df['Event_Time'] = pd.to_datetime(df['Event_Time'])

    # Create a new column 'group_by_key' combining the group_by_cols into a tuple
    df['group_by_key'] = df[group_by_cols].apply(tuple, axis=1)

    # Define the aggregation functions for each column
    agg_dict = {col: ['mean', 'std', lambda x: skew(x, nan_policy='omit'), lambda x: kurtosis(x, nan_policy='omit')] for col in agg_columns}

    # Perform the aggregation
    aggregated_df_mmt = df.groupby('group_by_key').agg(agg_dict)

    # Flatten the multi-level column index and rename the columns
    # aggregated_df.columns = [f"{col[0]}_{func}" for col in agg_columns for func in ['mean', 'std', 'skew', 'kurtosis']]
    aggregated_df_mmt.columns = [f"{'Sp' if col == 'Surprise' else col[0]}_{func}"
                             for col in agg_columns for func in ['mean', 'std', 'skew', 'kurtosis']]

    # regression features
    def beta(df, indp, dep):
      return np.cov(df[indp], df[dep])[0, 1] / np.var(df[indp])

    def resid_vol(df, indp, dep):
      beta = np.cov(df[indp], df[dep])[0, 1] / np.var(df[indp])
      resid = df[dep] - beta * df[indp]
      return np.std(resid)

    aggregated_df_regression = df.groupby('group_by_key').agg(
        Gold_beta=('Gold', lambda x: beta(df.loc[x.index], 'SP500', 'Gold')),
        Treasury10Y_beta=('Treasury10Y', lambda x: beta(df.loc[x.index], 'SP500', 'Treasury10Y'))
    )
    aggregated_df_regression.columns = [f"{col[0]}_{func}" for col in ['Treasury10Y', 'Gold'] for func in ['beta']]

    aggregated_df = pd.merge(aggregated_df_mmt, aggregated_df_regression, on = ['group_by_key'], how='left')

    # Reset the index to turn group-by columns back into normal columns
    aggregated_df.reset_index(inplace=True)

    # Split the 'group_by_key' tuples back into their original columns
    aggregated_df[group_by_cols] = pd.DataFrame(aggregated_df['group_by_key'].tolist(), index=aggregated_df.index)

    # Drop the 'group_by_key' column
    aggregated_df.drop(columns=['group_by_key'], inplace=True)

    # Delete the 'Surprise_std' column if it exists
    if 'Sp_std' in aggregated_df.columns:
        aggregated_df.drop(columns=['Sp_std'], inplace=True)
    if 'Sp_skew' in aggregated_df.columns:
        aggregated_df.drop(columns=['Sp_skew'], inplace=True)
    if 'Sp_kurtosis' in aggregated_df.columns:
        aggregated_df.drop(columns=['Sp_kurtosis'], inplace=True)

    # # Rename the 'Surprise_mean' column to 'Surprise'
    # aggregated_df.rename(columns={'Sp_mean': 'Surprise'}, inplace=True)

    cols = ['Event', 'Event_Time'] + [col for col in aggregated_df.columns if col not in ['Event', 'Event_Time']]
    aggregated_df = aggregated_df[cols]

    return aggregated_df.dropna()


In [7]:
# @title function: before_after_features
from tqdm import tqdm
def before_after_features(event_data, ts_data, group_by_cols, agg_columns, intervals=[-90, -75, -60, -45, -30, -15, 0, 15, 30, 45, 60, 75, 90], path='/content/drive/MyDrive/CFEM Greenmantle Project/Data/Data_adj/new_features_different_intervals/'):
    """
    This function calculates aggregated features for intervals before and after an event,
    renames the columns to include interval information in the format 'beforeX_afterY_Feature',
    and saves each interval's features as a CSV file. It also saves the final concatenated
    DataFrame with all intervals' features to the specified path.

    Parameters:
    event_data/ts_data : DataFrame
        Data containing event or time series information.
    group_by_cols : list
        Columns to group by (typically event and time).
    agg_columns : dict
        Dictionary specifying which columns to aggregate and the corresponding aggregation functions.
    intervals : list
        List of interval boundaries to split the time periods (before and after the event).
    path : str
        Folder path to save the CSV files.

    Returns:
    DataFrame
        A concatenated DataFrame with all interval-based features.
    """

    # Create the folder if it doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)

    merged_dfs = []  # List to store features for each interval

    # Iterate through the intervals and calculate features for each range
    for i in tqdm(range(len(intervals) - 1), desc="Processing Intervals"):
        window_start = intervals[i]
        window_end = intervals[i + 1]

        if window_end == 0:
            interval_label = f'before{abs(window_start)}_event'
        elif window_start == 0:
            interval_label = f'event_after{window_end}'
        elif window_end < 0:
            interval_label = f'before{abs(window_start)}_before{abs(window_end)}'
        elif window_start > 0:
            interval_label = f'after{abs(window_start)}_after{abs(window_end)}'
        else:
            interval_label = f'before{abs(window_start)}_after{abs(window_end)}'

        # Merge the data based on the current interval window
        merged_data = merge_data_around_event(event_data, ts_data, "M", window_start, window_end)

        # Aggregate the features for the current interval
        features = aggregate_features(merged_data, group_by_cols, agg_columns)

        # features_mmt = aggregate_features(merged_data, group_by_cols, agg_columns)
        # features_regression = regression_features(merged_data, group_by_cols)
        # features = pd.merge(features_mmt, features_regression, on = ['Event, Event_Time'], how='left')

        # Save the individual features dataframe as a CSV before renaming columns
        csv_filename = f'{interval_label}.csv'
        features.to_csv(os.path.join(path, csv_filename), index=False)
        print(f'Saved interval {interval_label} to {csv_filename}')

        # Rename the columns with the interval information
        # features.columns = [f'{interval_label}_{col}' if col not in group_by_cols and col != 'Surprise' else col for col in features.columns]
        features.columns = [f'{interval_label}_{col}' if col not in group_by_cols else col for col in features.columns]

        if i != 0:
          # Drop the specified columns
          features = features.drop(columns=group_by_cols)
          # features = features.drop(columns='Surprise')

        # Append the features to the list
        merged_dfs.append(features)

    # Concatenate all the interval dataframes along the columns
    final_df = pd.concat(merged_dfs, axis=1)

    final_df['Surprise'] = event['Surprise']

    # Save the concatenated dataframe as a CSV file
    final_csv_filename = 'concatenated_intervals.csv'
    final_df.to_csv(os.path.join(path, final_csv_filename), index=False)
    print(f'Saved concatenated intervals dataframe to {final_csv_filename}')

    return final_df

In [8]:
# @title 30min feature generation
import warnings
warnings.filterwarnings('ignore')

group_by_cols = ['Event', 'Event_Time']
# agg_columns = ['SP500', 'Gold', 'Treasury10Y', 'Surprise']
agg_columns = ['SP500', 'Gold', 'Treasury10Y']

features_path = '/content/drive/MyDrive/CFEM Greenmantle Project/Data/Data_adj/new_features_v2_different intervals/'
final_features = before_after_features(event, ts, group_by_cols, agg_columns, [-90, -60, -30, 0, 30, 60, 90], features_path)

print(final_features.head())
print(sorted(list(final_features.columns)))

final_features1 = final_features.dropna()
final_features1.to_csv(features_path+'features_all.csv', index=False)

Processing Intervals:   0%|          | 0/6 [00:27<?, ?it/s]


TypeError: '>=' not supported between instances of 'str' and 'int'

# Feature Analysis

## Visualizing functions

In [ ]:
# @title function: loadings analysis  (loadings_to_df)

def loadings_to_df(pca_loadings_df, features_names, period, pc_no):
  if isinstance(pca_loadings_df, pd.DataFrame):
    # pca_loadings is a pd.df (rolling part code)
    pca_loadings_period = pd.DataFrame(pca_loadings_df['loadings'].iloc[period])
    pca_loadings_period.columns = features_names
    loadings_series = pca_loadings_period.iloc[pc_no]

  else:
    # pca_loadings is a list of list (bootstrap part code)
    pca_loadings_period = pd.DataFrame(pca_loadings_df[period])
    pca_loadings_period.columns = features_names
    loadings_series = pca_loadings_period.iloc[pc_no]

  def convert_strings_to_ints(strings):
    result = []
    for s in strings:
      if s == 'nan' or s is np.nan:
        result.append(np.nan)
      elif s.startswith('event'):
        result.append(0)
      elif s.startswith('before'):
        result.append(-int(s.replace('before', '')) )
      elif s.startswith('after'):
        result.append(int(s.replace('after', '')))
      else:
          result.append(np.nan)
    return result

  start_list = []
  end_list = []
  method_list = []
  asset_list = []
  values_list = []

  # Iterate through each column name in the index of the series
  for col_name, value in loadings_series.items():
      if col_name == 'Surprise':
        start, end, method, asset = np.nan, np.nan, 'Surprise', 'Surprise'
      # elif col_name == 'Event' or col_name == 'Year':
      elif col_name == 'Event' or col_name == 'Event_Time':
        continue
      else:
        parts = col_name.split('_')
        start, end, asset, method = parts[0], parts[1], parts[2], parts[-1]

      # Append the parsed values to the respective lists
      start_list.append(start)
      end_list.append(end)
      method_list.append(method)
      asset_list.append(asset)
      values_list.append(value)

  # Create a DataFrame from the lists
  df = pd.DataFrame({
      'start': convert_strings_to_ints(start_list),
      'end': convert_strings_to_ints(end_list),
      'method': method_list,
      'asset': asset_list,
      'value': values_list
  })

  # Display the DataFrame
  return df

# loadings_df = loadings_to_df(pca_loadings_df,features_names, 0, 0)
# loadings_df

In [ ]:
# @title function: loadings analysis  (loadings_plot)

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

def loadings_plot(pca_loadings_df, features_names,period, pc_no):
  loadings_df = loadings_to_df(pca_loadings_df, features_names, period, pc_no)
  categorized_list = ['distance < -30' if x <= -30 else '-30 < distance < 30' if -30 < x <= 30 else 'distance > 30' if not pd.isna(x) else np.nan for x in loadings_df['start']]
  # loadings_df['start_cat'] = np.ceil(loadings_df['start']/30)
  loadings_df['start_cat'] = categorized_list
  df_sorted = loadings_df.sort_values(by='value', ascending=False)

  fig, axs = plt.subplots(3, 1, figsize=(8, 8))

  unique_assets = loadings_df['asset'].unique()
  asset_colors = dict(zip(unique_assets, list(mcolors.TABLEAU_COLORS.values())[1:1+len(unique_assets)]))
  asset_colors['Surprise'] = 'black'

  method_colors = {'mean': 'red', 'std': 'blue', 'Surprise': 'black', 'kurtosis': 'green', 'skew': 'yellow', 'beta': 'orange'}

  unique_time = loadings_df['start_cat'].dropna().unique()
  time_colors = dict(zip(unique_time, list(mcolors.TABLEAU_COLORS.values())[:len(unique_time)]))

  def get_color_td(row):
    norm = Normalize(vmin=0, vmax=100)
    if row['method'] == 'Surprise':
      return 'black'
    elif row['method'] == 'mean':
      return plt.cm.Reds(norm(np.abs(row['start'])))
    elif row['method'] == 'std':
      # return plt.cm.Blues(norm(row['start']))
      return 'blue'
    elif row['method'] == 'kurtosis':
      return 'green'
    elif row['method'] == 'skew':
      return 'yellow'
    elif row['method'] == 'beta':
      return 'orange'
    return 'gray'

  def get_color_td_new(row):
    if row['method'] == 'Surprise':
      return 'black'
    else:
      return time_colors[(row['start_cat'])]

  def get_color_asset(row):
    if row['method'] == 'Surprise':
      return 'black'
    if row['method'] == 'std':
      return 'blue'
    if row['method'] == 'mean':
      return asset_colors[row['asset']]
    if row['method'] == 'kurtosis':
      return 'green'
    if row['method'] == 'skew':
      return 'yellow'
    if row['method'] == 'beta':
      return 'orange'
    return 'gray'

  def get_color_asset_new(row):
    if row['method'] == 'Surprise':
      return 'black'
    return asset_colors[row['asset']]

  # plot colormapped by method (mean/std)
  bars_method = axs[0].bar(range(len(loadings_df)),df_sorted['value'], color=[method_colors[m] for m in df_sorted['method']])
  axs[0].set_xlabel('Features')
  axs[0].set_ylabel('Loadings')
  axs[0].set_title(f'PC {pc_no+1} Loadings, Colormapped by Method')
  handles_method = [plt.Rectangle((0,0),1,1, color=method_colors[method]) for method in ['mean', 'std','Surprise', 'kurtosis', 'skew', 'beta']]
  axs[0].legend(handles_method, ['mean', 'std', 'Surprise', 'kurtosis', 'skew', 'beta'], title="Method", bbox_to_anchor=(1.05, 1), loc='upper left')
  axs[0].grid()
  # axs[0].set_xticks(range(len(df_sorted)))
  # axs[0].set_xticklabels(df_sorted['asset'], rotation=45, ha='right')

  # plot colormapped by asset (total: 3)
  bars_asset = axs[1].bar(range(len(df_sorted)), df_sorted['value'], color=[asset_colors[asset] for asset in df_sorted['asset']])
  axs[1].set_xlabel('Features')
  axs[1].set_ylabel('Loadings')
  axs[1].set_title(f'PC {pc_no+1} Loadings, Colormapped by Asset')
  handles_asset = [plt.Rectangle((0,0),1,1, color=asset_colors[asset]) for asset in unique_assets]
  axs[1].legend(handles_asset, unique_assets, title="Assets", bbox_to_anchor=(1.05, 1), loc='upper left')
  axs[1].grid()
  # axs[1].set_xticks(range(len(df_sorted)))
  # axs[1].set_xticklabels(df_sorted['asset'], rotation=45, ha='right')

  # plot colormapped by method & time distance
  bars_time = axs[2].bar(range(len(df_sorted)), df_sorted['value'], color=df_sorted.apply(get_color_td_new, axis=1))
  axs[2].set_xlabel('Features')
  axs[2].set_ylabel('Loadings')
  axs[2].set_title(f'PC {pc_no+1} Loadings, Colormapped by Time Distance')
  handles_time = [plt.Rectangle((0,0),1,1, color=time_colors[time]) for time in unique_time]
  handles_time.append(plt.Rectangle((0,0),1,1, color='black'))
  axs[2].legend(handles_time, list(unique_time)+['Surprise'], title="Time distance", bbox_to_anchor=(1.05, 1), loc='upper left')
  axs[2].grid()

  # # plot colormapped by method & asset
  # bars_assetmethod = axs[2].bar(range(len(df_sorted)), df_sorted['value'], color=df_sorted.apply(get_color_asset, axis=1))
  # axs[2].set_xlabel('Features')
  # axs[2].set_ylabel('Loadings')
  # axs[2].set_title(f'PC {pc_no+1} Loadings, Colormapped by Asset Mean')
  # handles_assetmethod = [plt.Rectangle((0,0),1,1, color=asset_colors[asset]) for asset in unique_assets]
  # handles_assetmethod.append(plt.Rectangle((0,0),1,1, color='blue'))
  # axs[2].legend(handles_assetmethod, list(unique_assets)+['std'], title="Assets Mean", bbox_to_anchor=(1.05, 1), loc='upper left')
  # axs[2].grid()

  if isinstance(pca_loadings_df, pd.DataFrame):
    start_year = pca_loadings_df.iloc[period]['start_year']
    end_year = pca_loadings_df.iloc[period]['end_year']
    axs[0].set_title(f'PC {pc_no+1} Loadings ({start_year} - {end_year}), Colormapped by Method')
    axs[1].set_title(f'PC {pc_no+1} Loadings ({start_year} - {end_year}), Colormapped by Asset')
    axs[2].set_title(f'PC {pc_no+1} Loadings ({start_year} - {end_year}), Colormapped by Time Distance')
    # axs[2].set_title(f'PC {pc_no+1} Loadings ({start_year} - {end_year}), Colormapped by Asset Mean')


  plt.tight_layout()
  plt.show()

# loadings_plot(pca_loadings_df,features_names, 0, 0)

In [ ]:
# @title function: loadings analysis  (with names)

def loadings_plot_names(pca_loadings_df, features_names,period, pc_no):
  loadings_df = loadings_to_df(pca_loadings_df, features_names, period, pc_no)
  df_sorted = loadings_df.sort_values(by='value', ascending=False)

  # fig, axs = plt.subplots(1, 1, figsize=(12, 10))
  plt.figure(figsize=(15, 8))

  method_colors = {'mean': 'red', 'std': 'blue', 'Surprise': 'black'}

  # string_list = df_sorted.apply(lambda row: f"({int(row['start'])}) {row['asset']}", axis=1).tolist()

  string_list = df_sorted.apply(
    lambda row: f"({int(row['start'])}) {row['asset']} _{row['method']}" if not pd.isna(row['start']) else row['asset'], axis=1
  ).tolist()

  # plot colormapped by method (mean/std)
  bars_method = plt.bar(range(len(loadings_df)), df_sorted['value'], color=[method_colors[m] for m in df_sorted['method']])
  plt.xlabel('Features')
  plt.ylabel('Loadings')
  plt.title(f'PC {pc_no+1} Loadings, Colormapped by Method')

  # Create legend handles
  handles_method = [plt.Rectangle((0, 0), 1, 1, color=method_colors[method]) for method in ['mean', 'std', 'Surprise']]
  plt.legend(handles_method, ['mean', 'std', 'Surprise'], title="Method", bbox_to_anchor=(1.05, 1), loc='upper left')

  plt.grid(True)
  plt.xticks(ticks=range(len(string_list)), labels=string_list, rotation=90)  # Ensure xticks aligns with string_list
  plt.tight_layout()  # Adjust layout to prevent overlap
  plt.show()

# loadings_plot(pca_loadings_df,features_names, 0, 0)

## Global PCA for features

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import to_rgba

from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
features_path = '/content/drive/MyDrive/CFEM Greenmantle Project/Data/Data_adj/new_features_different intervals/concatenated_intervals.csv'
features = pd.read_csv(features_path)

In [ ]:
# @title Heatmap for features
import seaborn as sns
plt.figure(figsize=(12, 12))

features_names = features.columns.drop(['Event', 'Event_Time'])
corr_matrix = features[features_names].corr()
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)

sns.clustermap(corr_matrix, cmap='coolwarm', figsize=(12, 10), annot=False)

# Add title
plt.title('Clustermap of Correlations')

plt.show()

In [ ]:
# @title Global PCA
X = features[features_names]
X_normalized = (X - X.mean()) / X.std()
pca = PCA()
pca.fit(X_normalized)

cumvar_global = np.cumsum(pca.explained_variance_ratio_)
loadings_global = pca.components_

pca_cumvars_df = pd.DataFrame(cumvar_global)
pca_cumvars_df

In [ ]:
# @title Scree Plot

import matplotlib.pyplot as plt

plt.figure(figsize = (10,5))
plt.plot(range(len(cumvar_global)), cumvar_global, '--', label='Global')
plt.scatter(range(len(cumvar_global)), cumvar_global, marker='o', label='Global')

# Add labels and title
plt.grid()
plt.xlabel('Principal Component')
plt.ylabel('Cumulative Explained Variance')
_ = plt.title('Distribution of Cumulative Explained Variance')


In [ ]:
# @title PC 1 Loadings (with names)
features_names = features.columns.drop(['Event', 'Event_Time'])
loadings_plot_names([loadings_global], features_names, 0, 0)

In [ ]:
# @title PC 2 Loadings (with names)
features_names = features.columns.drop(['Event', 'Event_Time'])
loadings_plot_names([loadings_global], features_names, 0, 1)

In [ ]:
# @title PC 3 Loadings (with names)
features_names = features.columns.drop(['Event', 'Event_Time'])
loadings_plot_names([loadings_global], features_names, 0, 2)

In [ ]:
# @title PC 1 Loadings (Colormapped to show phenomenon)
features_names = features.columns.drop(['Event', 'Event_Time'])
loadings_plot([loadings_global], features_names, 0, 0)

In [ ]:
# @title PC 2 Loadings (Global)
features_names = features.columns.drop(['Event', 'Event_Time'])
loadings_plot([loadings_global], features_names, 0, 1)

In [ ]:
# @title PC 3 Loadings (Global)
features_names = features.columns.drop(['Event', 'Event_Time'])
loadings_plot([loadings_global], features_names, 0, 2)

## Bootstrap PCA for features (for stability)

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import to_rgba

from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
features_path = '/content/drive/MyDrive/CFEM Greenmantle Project/Data/Data_adj/new_features_30different intervals/concatenated_intervals.csv'
features = pd.read_csv(features_path)

In [ ]:
# @title Bootstrap PCA

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.utils import resample
from tqdm import tqdm

np.random.seed(42)

# Parameters
bootstrap_size = 4000
iterations = 15
cumvar_list = []
loadings_list = []

for i in tqdm(range(iterations), desc="Bootstrap PCA iterations"):
    # Bootstrap sampling with replacement
    # bootstrap_sample = resample(features.drop(['Event', 'Year'], axis=1), n_samples=bootstrap_size, replace=True)
    bootstrap_sample = resample(features.drop(['Event', 'Event_Time'], axis=1), n_samples=bootstrap_size, replace=True)
    bootstrap_sample = bootstrap_sample.dropna()
    X_normalized = (bootstrap_sample - bootstrap_sample.mean()) / bootstrap_sample.std()
    pca = PCA()
    pca.fit(X_normalized)

    # Store cumulative variance and factor loadings
    cumvar_list.append(np.cumsum(pca.explained_variance_ratio_))
    loadings_list.append(pca.components_)

# Saving the results to dataframes
pca_cumvars_df = pd.DataFrame(cumvar_list)

In [ ]:
# @title Scree Plot (boxplotting for multiple iterations)

import matplotlib.pyplot as plt

plt.figure(figsize = (10,5))

# plt.boxplot(pca_cumvars_df.iloc[:,0:15])
plt.boxplot(pca_cumvars_df)

means = [np.average(pca_cumvars_df.iloc[:,pc]) for pc in range(len(pca_cumvars_df.columns))]
# means = [np.average(pca_cumvars_df.iloc[:,pc]) for pc in range(15)]
plt.plot(range(1, len(means) + 1), means, '--', label='Mean')

# Add labels and title
plt.grid()
plt.xlabel('Principal Component')
plt.ylabel('Cumulative Explained Variance')
_ = plt.title('Distribution of Cumulative Explained Variance')


In [ ]:
# @title PC 1 Loadings (Iteration 1, Colormapped to show phenomenon)
features_names = features.columns.drop(['Event', 'Event_Time'])
loadings_plot(loadings_list, features_names, 0, 0)




## Yifu's Analysis

In [ ]:
# @title Feature Standarization
z_feature = (features - features.mean()) / features.std()

In [ ]:
# @title Feature PCA cumulative explanation
pca = PCA()
principal_components = pca.fit_transform(z_feature)

explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)
components = np.arange(1, len(explained_variance) + 1)

plt.figure(figsize=(10, 6))

# Plot the explained variance as bars
plt.bar(components, explained_variance, alpha=0.6, label='Explained Variance per Component')

# Plot the cumulative variance
plt.plot(components, cumulative_variance, marker='o', linestyle='--', color='red', label='Cumulative Explained Variance')

plt.title('Explained Variance by PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.xticks(components)  # Ensure x-ticks correspond to component numbers
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# @title Feature PCA with appropriate number of PC
n_components = 7
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(z_feature)
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loading_df = pd.DataFrame(loadings, index=z_feature.columns, columns=[f'PC{i+1}' for i in range(n_components)])

In [ ]:
# @title Adjust i for the component exposure plot
i = 0
plt.figure(figsize=(8, 6))

# Extract the component loadings and sort by values
component_loadings = loading_df.iloc[:, i].sort_values(ascending=False)

# Plot the sorted loadings
component_loadings.plot(kind='bar')

plt.title(f'Loadings for Principal Component {i+1}')
plt.ylabel('Loading')
plt.xticks(rotation=90)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# Finalized Feature Generation

In [ ]:
def feature_output(feature_df):
  z_feature = feature_df.copy()
  vol_col = [col for col in z_feature.columns if 'std' in col]
  ret_col = [col for col in z_feature.columns if 'mean' in col]
  skew_col = [col for col in z_feature.columns if 'skew' in col]
  kurt_col = [col for col in z_feature.columns if 'kurtosis' in col]
  beta_col = [col for col in z_feature.columns if 'beta' in col]
  before_ret = [col for col in ret_col if 'before' in col]
  after_ret = [col for col in ret_col if 'after' in col]
  before_vol = [col for col in vol_col if 'before' in col]
  after_vol = [col for col in vol_col if 'after' in col]
  before_skew = [col for col in skew_col if 'before' in col]
  after_skew = [col for col in skew_col if 'after' in col]
  before_kurt = [col for col in kurt_col if 'before' in col]
  after_kurt = [col for col in kurt_col if 'after' in col]
  before_beta = [col for col in beta_col if 'before' in col]
  after_beta = [col for col in beta_col if 'after' in col]

  final = pd.DataFrame()
  final['suprise'] = z_feature['Surprise']
  final['vol shift'] = z_feature[after_vol].mean(axis=1)- z_feature[before_vol].mean(axis=1)
  final['ret shift'] = z_feature[after_ret].mean(axis=1) - z_feature[before_ret].mean(axis=1)
  final['skew shift'] = z_feature[after_skew].mean(axis=1) - z_feature[before_skew].mean(axis=1)
  final['beta shift'] = z_feature[after_beta].mean(axis=1) - z_feature[before_beta].mean(axis=1)
  final['kurt shift'] = z_feature[after_kurt].mean(axis=1) - z_feature[before_kurt].mean(axis=1)
  return final